In [ ]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd = sc.parallelize(splitedRdd.take(10000))
splitedRddSample1 = sc.parallelize(splitedRdd.takeSample(True, 10000,0))
splitedRddSample2 = sc.parallelize(splitedRdd.takeSample(True, 10000,1))
splitedRddSample1.take(1)
#(uid,[[train],[test]]

In [ ]:
numRecList = 5
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores
def evaluation_rr1(testItem, recList):
    scores = 0
    for i in range(len(recList)):
        if testItem == recList[i]:
            scores = 1.0 / (i+1)
            break
    return scores
def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores  

def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation2.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()
    
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation2.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[3]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.close()
def print_result3(finalScore, algorithm_name):
    f = open('result/mfu_test_result1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()
def print_result4(finalScore, algorithm_name):
    f = open('result/mfu_test_result2.txt','a')
    f.write('10000 users with duplicated evaluations with pop apps removed')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [12]:
from collections import Counter
def mfuFunction_P(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    
    for i in range(numTest):  
        #testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision

        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_precision = scores_precision / numTest

    return scores_precision
finalScore = splitedRdd.map(mfuFunction_P)
print_result1(finalScore,'mfuFunction_P')

In [10]:
from operator import add
numRecList = 5
def lruFunction_P(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    for i in range(numTest):
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        testList_items = [t[0] for t in testList]                      #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > numRecList-1:
            finalRecommender = Recommender[:numRecList]
        else:    
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender

        scores_precision = evaluation_precision(testList_items[0], finalRecommender) + scores_precision
        trainList = sorted(trainList,key=lambda x:int(x[1][0]));
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_precision = scores_precision / numTest

    return scores_precision

finalScore = splitedRdd.map(lruFunction_P)
#finalScore.map(lambda x:x[2][0]).mean()
#finalScore.take(10)#,finalScore.map(lambda x:x[2]).mean()
print_result1(finalScore, 'lruFunction_P')

In [9]:
def mruFunction_P(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_precision = evaluation_precision(testList_items[0], finalRecommender) + scores_precision
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_precision = scores_precision / numTest

    return scores_precision

finalScore = splitedRdd.map(mruFunction_P)
print_result1(finalScore,'mruFunction_P')

In [77]:
def numberOfApps(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    numApps = len(remove_duplicates([x[0] for x in trainList]))
    
    #numTrain = len(trainList)
    return line[0],line[1],numApps
splitedRDDFilter2 = splitedRdd.map(numberOfApps).filter(lambda x:x[2]<20).map(lambda x:(x[0],x[1]))
#splitedRDDFilter2 = splitedRddv2.map(numberOfApps)
splitedRDDFilter2.count()

2286

In [ ]:
from operator import add
numRecList = 5
def lruFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rap = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        testList_items = [t[0] for t in testList]                      #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > numRecList-1:
            finalRecommender = Recommender[:numRecList]
        else:    
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0]));
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRdd.map(lruFunction)
#finalScore.map(lambda x:x[2][0]).mean()
#finalScore.take(10)#,finalScore.map(lambda x:x[2]).mean()
print_result2(finalScore, 'lru')

In [28]:
from operator import add
numRecList = 5
def lruFunction_filter(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr1 = 0.0
    scores_rap = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        testList_items = [t[0] for t in testList]                      #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > numRecList-1:
            finalRecommender = Recommender[:numRecList]
        else:    
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0]));
        trainList.append(testList[0])
        testList.pop(0)   
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap

finalScore = splitedRDDFilter2.map(lruFunction_filter)
#finalScore.map(lambda x:x[2][0]).mean()
#finalScore.take(10)#,finalScore.map(lambda x:x[2]).mean()
print_result3(finalScore, 'lruFunction_filter')

In [ ]:
def mruFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_rr = 0.0
    scores_ap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        trainList.append(testList[0])
        testList.pop(0)  
        
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]

    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(mruFunction)
print_result(finalScore,'mru')

In [72]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(mfuFunction)
print_result3(finalScore,'mfu')

In [75]:
from collections import Counter
def mfuFunction_no_iterative(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    testList_items = [t[0] for t in testList]                #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    Recommender = Counter(RecommenderDuplicate).most_common()
    Recommender =  [t[0] for t in Recommender] 

    if len(Recommender) > 4:
        finalRecommender = Recommender[:5]
    else:
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(Recommender)
        finalRecommender[:numRec] = Recommender
    scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
    scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
    scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
    scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
    scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
    scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(mfuFunction_no_iterative)
print_result3(finalScore,'mfuFunction_no_iterative')

In [25]:
def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

a = [2,3,4,5,1]
b = [1,22,3,14,15]
evaluation_rap(a,b)

0.8333333333333333

In [68]:
from collections import Counter
def mfuFunction_time(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    #trainList = trainList[(numTrain*39/40):numTrain]
#     trainList.extend(testList[:(numTest/2)])
#     numTrain = len(trainList)
#     testList = testList[(numTest/2):]
#     numTest = len(testList)
    trainList.extend(testList)
    numTrain = len(trainList)
    testList = [x for x in trainList if trainList[numTrain-1][1][0] - x[1][0]<=43200]
    numTest = len(testList)
    trainList = trainList[:(numTrain-numTest)]
    scores_precision = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        while (len(trainList)>0 and testList[0][1][0]-trainList[0][1][0]>86400):
            trainList.pop(0)
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        #trainList.pop(0)
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(mfuFunction_time)
print_result2(finalScore,'mfuFunction_time')

In [49]:
from operator import add
from collections import Counter
f = open('result/mfu_test_result1.txt','a')
f.write('-------mfu giving weight-----')
f.write('\n')
f.close()
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
#        testcount = Counter(testList_items).most_common()
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        Recommender = [[t[0],t[1]*5] if t[0] in Recommender1[:10] else t for t in Recommender]
#         Recommender = [[t[0],1.0*t[1]/numTrain] for t in Recommender]
#         testcount = [[t[0],1.0*t[1]/numTest] for t in testcount]
       
    #weighted
#         Recommender =  [[t,0] for t in Recommender] 
#         for i in range(len(Recommender)):
#             for j in range(len(trainList)):
#                 if trainList[j][0] == Recommender[i][0]:
#                     if len(trainList)>0 and (trainList[-1][1][0] - trainList[j][1][0]) == 0:
#                         weight = 1
#                     elif len(trainList)>0:
#                         weight = (trainList[-1][1][0] - trainList[j][1][0])/86400
#                     else:
#                         weight = 0
#                     if weight != 0:
#                         Recommender[i][1] = Recommender[i][1] + 1.0/weight
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = splitedRdd.map(mfuFunction)
#finalScore.take(5)
print_result3(finalScore,'mfu')

In [38]:
from operator import add
from collections import Counter
def removePopApps(line):
    listGroup = line[1]
    top = [t[0] for t in Counter([t[0] for t in listGroup[0]]  + [t[0] for t in listGroup[1]] ).most_common()][:15]  #top5
    
    listGroup[0] = [t for t in listGroup[0] if t[0] not in top]      #0.8 train set
    listGroup[1] = [t for t in listGroup[1] if t[0] not in top]       #0.2 test set
    return line[0],listGroup

splitedRdd1 = splitedRddSample1.map(removePopApps)
splitedRdd2 = splitedRddSample2.map(removePopApps)
splitedRdd2.take(1)

[(7652,
  [[train(itemId=79314, context=context(ts=1420997495, city=2054, lat=39.928886, lon=-75.322601, moving=2, location=2, time_of_day=3)),
    train(itemId=57365, context=context(ts=1420998086, city=2054, lat=39.928864, lon=-75.322571, moving=1, location=2, time_of_day=3)),
    train(itemId=57365, context=context(ts=1420998769, city=2054, lat=39.928928, lon=-75.322525, moving=1, location=2, time_of_day=3)),
    train(itemId=2766, context=context(ts=1421006011, city=2054, lat=39.92889, lon=-75.322609, moving=2, location=3, time_of_day=3)),
    train(itemId=8509, context=context(ts=1421012576, city=2054, lat=39.928871, lon=-75.322525, moving=1, location=3, time_of_day=4)),
    train(itemId=15939, context=context(ts=1421021887, city=2054, lat=39.928848, lon=-75.322563, moving=2, location=3, time_of_day=4)),
    train(itemId=15939, context=context(ts=1421022391, city=2054, lat=39.929054, lon=-75.322777, moving=2, location=3, time_of_day=4)),
    train(itemId=77050, context=context(ts=

In [39]:
from operator import add
from collections import Counter
f = open('result/mfu_test_result2.txt','a')
f.write('-------mfu only recommend to 5 test-----')
f.write('\n')
f.close()
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    
#     top = [t[0] for t in Counter([t[0] for t in listGroup[0]]  + [t[0] for t in listGroup[1]] ).most_common()][:5]  #top5
    
    trainList = [t for t in listGroup[0]]      #0.8 train set
    testList = [t for t in listGroup[1]]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
#        testcount = Counter(testList_items).most_common()
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        Recommender = [[t[0],t[1]*5] if t[0] in Recommender1[:10] else t for t in Recommender]
#         Recommender = [[t[0],1.0*t[1]/numTrain] for t in Recommender]
#         testcount = [[t[0],1.0*t[1]/numTest] for t in testcount]
       
    #weighted
#         Recommender =  [[t,0] for t in Recommender] 
#         for i in range(len(Recommender)):
#             for j in range(len(trainList)):
#                 if trainList[j][0] == Recommender[i][0]:
#                     if len(trainList)>0 and (trainList[-1][1][0] - trainList[j][1][0]) == 0:
#                         weight = 1
#                     elif len(trainList)>0:
#                         weight = (trainList[-1][1][0] - trainList[j][1][0])/86400
#                     else:
#                         weight = 0
#                     if weight != 0:
#                         Recommender[i][1] = Recommender[i][1] + 1.0/weight
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        if len(testList_items)>5:
            testList_items = testList_items[:5]
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = splitedRddSample1.map(mfuFunction)
print_result4(finalScore,'mfu_sample1')
finalScore = splitedRddSample2.map(mfuFunction)
#finalScore.take(5)
print_result4(finalScore,'mfu_sample2')

In [1]:
f = open('result/mfu_test_result2.txt','a')
f.write('-------bayesian_moving only recommend to 5-----')
f.write('\n')
f.close()
def bayesian_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext # /numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi   #/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set
        
        if len(testList_items)>5:
            testList_items = testList_items[:5]
        
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = splitedRddSample1.map(bayesian_moving)
print_result4(finalScore, 'bayesian_moving_sample1')
finalScore = splitedRddSample2.map(bayesian_moving)
print_result4(finalScore, 'bayesian_moving_sample2')

NameError: name 'splitedRddSample1' is not defined